In [23]:
import base64

In [12]:
import binascii

In [ ]:
code ="mvRJQ09RNOK4hP+rHwPPWUATjzj4O5AsUctIUE3tJsyRz2AHfSIp+Y4Ex0iE0/I6xwM7kRRIgICEXQX/S9wAtgw31jVeHkd44cWUmYWGMB0AG9t4IQ0cQDGplgs1I4ONhkPr3l5UulVJCAlEmMKbjXWbc2iTiF/kJcnpsgCF+Sk="

In [21]:
code ='YXR0YWNrIGF0IGRhd24h '

In [ ]:
hex_code = '9af4C4e2b884ffab1f03cf138f8f890cbedcc91cf07f98e04c784d3f2c703911480808405ffdc00b60c7d651ee1c59499858601d001bdb1c1a9960b5838d86Cebdeba08D98c29b8d9b9388e4c9e9b20085f9'

In [24]:
decoded_data = base64.b64decode(code)

In [25]:
decoded_data

b'attack at dawn!'

In [ ]:
decoded_text = bytes.fromhex(hex_code).decode('utf-8')
print(decoded_text)

UnicodeDecodeError: ignored

In [ ]:

encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252', 'utf-16']

for encoding in encodings:
    try:
        decoded_text = bytes.fromhex(hex_code).decode(encoding)
        print(f"Decoded using {encoding}: {decoded_text}")
    except UnicodeDecodeError:
        print(f"Decoding with {encoding} failed")

Decoding with utf-8 failed
Decoded using latin-1: ôÄâ¸ÿ«Ï¾ÜÉðàLxM?,p9H@_ýÀ`ÇÖQîYIX` Û`µÎ½ë ÂäÉé² ù
Decoded using iso-8859-1: ôÄâ¸ÿ«Ï¾ÜÉðàLxM?,p9H@_ýÀ`ÇÖQîYIX` Û`µÎ½ë ÂäÉé² ù
Decoding with cp1252 failed
Decoding with utf-16 failed




---



In [3]:
pip install pycryptodome


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.9 MB/s eta 0:00:00


In [20]:
"""[MIF29] TP-Oracle : attaque à l'oracle de parité

Voir <http://mif29-crypto-sec.pages.univ-lyon1.fr/TP-oracle/>
"""

import binascii
import math
from pathlib import Path

import requests
from Crypto.PublicKey import RSA

HOST = "192.168.239.59"
PORT = 8080
URL = f"http://{HOST}:{PORT}/parity/"

URL_EXAMPLE = "http://mif29-crypto-sec.pages.univ-lyon1.fr/TP-oracle/example.b64"
KEY_SIZE = 2**10

CIPHERTEXT_FILE = Path("ciphertext.b64")
KEYFILE_PUB = Path("rsa_parity_public.pem")

# objet global pour les requêtes HTTP
session = requests.Session()


## Préparation : téléchargement si nécessaire du message et de la clef publique

def download_content(filepath, uri):
    """Download HTTP content into a file"""
    if not filepath.exists():
        req = requests.get(uri, timeout=100000000.0)
        with open(filepath, "wb") as f:
            f.write(req.content)
            print(f"File {filepath} written (first download)")

download_content(CIPHERTEXT_FILE, URL + "ciphered")
download_content(KEYFILE_PUB, URL + "pubkey")

## Chargement du message et de la clef publique
with open(CIPHERTEXT_FILE, "rb") as file:
    original_ciphertext = file.read()
    print(f"File {CIPHERTEXT_FILE} read")

with open(KEYFILE_PUB, "r", encoding="ascii") as file:
    public_key = RSA.import_key(file.read())
    print(f"File {KEYFILE_PUB} read ({public_key.e=})")

## helpers
def get_parity(message: bytes) -> bool:
    """get parity of cleartext from b64 encoded message, True iff cleartext is odd"""
    res = session.post(URL, data=message, timeout=1.0)
    if res.status_code not in (200,):
        raise ValueError(f"{res.status_code} ({res.reason}): {res.content}")
    if res.content not in (b"Even", b"Odd"):
        raise ValueError(f"Unknown answer {res.content}")
    return res.content == b"Odd"

def encrypt(pubkey, m: bytes):
    """textbook RSA cipher: takes a b64 message, encrypt then return b64"""
    n, e = pubkey.n, pubkey.e
    msg = int.from_bytes(binascii.a2b_base64(m), "big", signed=False)
    if msg > n:
        raise ValueError(f"Cleartext {msg.bit_length()} bits is too long")
    ciphered = pow(msg, e, n)
    size = math.ceil(ciphered.bit_length() / 8)
    return binascii.b2a_base64(ciphered.to_bytes(size, "big"), newline=False)

def multiply_cleartext_by_2(pubkey, m: bytes):
    """read b64, converts to int, multiply by 2 then convert back to b64"""
    n, e = pubkey.n, pubkey.e
    msg = int.from_bytes(binascii.a2b_base64(m), "big", signed=False)
    msg =  (pow(2, e, n) * msg) % n
    size = math.ceil(msg.bit_length() / 8)
    return binascii.b2a_base64(msg.to_bytes(size, "big"), newline=False)


def find_message(pubkey, base_message: bytes):
    """Dichomoty to find original message bit by bits"""
    # intervalle de départ [low, high[
    n = pubkey.n
    low, high = 0, n
    message = base_message
    for i in range(900):
        message = multiply_cleartext_by_2(pubkey, message)
        mid = (high + low) // 2
        high = mid
    s = 1
    while (high - low) > 2**4:
        lowsize = math.ceil(low.bit_length() / 8)
        highsize = math.ceil(high.bit_length() / 8)
        print(low.to_bytes(lowsize, "big"))
        print(high.to_bytes(highsize, "big"))
        print(f"{s=}")
        s +=1
        message = multiply_cleartext_by_2(pubkey, message)
        mid = (high + low) // 2
        if (get_parity(message) == 1): # 2*m > n
            print("2*m > n")
            low = mid
        else: # 2*m < n
            print("2*m < n")
            high = mid
    return low, high

if __name__ == "__main__":
    temp = "mvRJQ09RNOK4hP+rHwPPWUATjzj4O5AsUctIUE3tJsyRz2AHfSIp+Y4Ex0iE0/I6xwM7kRRIgICEXQX/S9wAtgw31jVeHkd44cWUmYWGMB0AG9t4IQ0cQDGplgs1I4ONhkPr3l5UulVJCAlEmMKbjXWbc2iTiF/kJcnpsgCF+Sk=".encode("utf-8")
    low, high = find_message(public_key, temp)
    for i in range(low-2**6, high+2**6):
        isize = math.ceil(i.bit_length() / 8)
        i_base64 = binascii.b2a_base64(i.to_bytes(isize, "big"), newline=False)
        ciphered_i_base64 = encrypt(public_key, i_base64)
        if (temp == ciphered_i_base64):
            print(f"string : {i.to_bytes(isize, 'big')}")
            print(ciphered_i_base64)

ConnectTimeout: ignored